# Chapter 3: Working with APIs

[Overview of what APIs are]

Most of the APIs of interest to social scientists weren't designed for our use. They are primarily for web or mobile app developers who want to include the content on their pages. So while I might use the [MapQuest](http://developer.mapquest.com) API to look at how often intra-city trips involve highways, the target audience is business owners trying to help people get to their store. Similarly, scores of researchers have used data from [Twitter APIs](https://dev.twitter.com/docs/api/1.1) to study politics, but it was developed so that you could put a custom Twitter widget on your home page. 

The good news is that since these services want you to use their data, the APIs are often well documented, especially for languages like Python that are popular in Silicon Valley. The bad news is that APIs don't always make available the parts of the service, like historical data, that are of most interest to researchers. The worst news is research using APIs frequently violates the providers Terms of Service, so it can be an ethical grey zone. 

### A first API

When you sign up as a developer to use an API, you usually agree to only use the API to facilitate other people using the service (e.g. customer's finding their way to your store) and that you won't store the data. API providers usually enforce this through rate limiting, meaning you can only access the service so many times per minute or per day. For example, you can only search status updates 180 times every 15 minutes according to [Twitter guidelines](https://dev.twitter.com/docs/rate-limiting/1.1/limits). [Yelp](http://www.yelp.com/developers/documentation/faq) limits you to 10,000 calls per day. If you go over your limit, you won't be able to access the service for a bit. You will also get in trouble if you redistribute the data, so don't plan on doing that. One API developed specially for researcher is available ... [Chronicling America](http://chroniclingamerica.loc.gov/about/) is a joint project of the National Endowment for the Humanities and the Library of Congress that [More description.

The website has a search function and my search for the term "[slavery](http://chroniclingamerica.loc.gov/search/pages/results/?andtext=slavery)" returned 404,325 results.

<img src="images/ca_slavery_search.png">

This is great, but they make it much better for researchers by providing an API to assist with searching and downloading their archive. [Note about bulk downloads.]

One of the nice things about APIs is that they are often intuitive, or at least interpretable after you see them. For example, to retrieve the first page of search results into an easily digestable format, you append ``&format=json`` to the end of the search URL ``http://chroniclingamerica.loc.gov/search/pages/results/?andtext=slavery``. In your browser, this returns a text file in the JSON format.

<img src="images/ca_slavery_api.png">

Thankfully, the programers have made the variable names understandable. As before, the search found 404,325 results (``"totalItems": 404325,``). The server did not return all of these, however, just 20 of them (``"itemsPerPage": 20``), starting with the first result (``"startIndex": 1,``) and ending with the 20th (``"endIndex": 20,``). 


http://chroniclingamerica.loc.gov/search/pages/results/?andtext=slavery&format=json


(http://chroniclingamerica.loc.gov/about/api/)

[``requests``](http://docs.python-requests.org/en/master/) is a useful and commonly used HTTP library for python. It is not a part of the default installation, but is included with Anaconda Python Distribution. 

In [30]:
import requests

It would be possible to use the API URL and parameters directly in the requests command, but since the most likely scenario involves making repeating calls to ``requests`` as part of a loop -- the search returned less than 1% of the results -- I store the strings first. 

In [31]:
base_url = 'http://chroniclingamerica.loc.gov/search/pages/results/'
parameters = '?andtext=slavery&format=json'

`requests.get()` is used for both accessing websites and APIs. The command can be modified by several arguements, but at a minimum, it requires the URL.

In [32]:
r = requests.get(base_url + parameters)

`r` is a `requests` response object. Any JSON returned by the server are stored in `.json().`

In [33]:
search_json = r.json()

JSONs are dictionary like objects, in that they have keys (think variable names) and values. `.keys()` returns a list of the keys.

In [34]:
print search_json.keys()

[u'totalItems', u'endIndex', u'startIndex', u'itemsPerPage', u'items']


You can return the value of any key by putting the key name in brackets.

In [35]:
search_json['totalItems']

404549

As is often the case with results from an API, most of the keys and values are metadate about either the search or what is being returned. These are useful for knowing if the search is returning what you want, which is particularly important when you are making multiple calls to the API. 

The data I'm intereted in is all in `items`. 

In [36]:
print type(search_json['items'])
print len(search_json['items'])

<type 'list'>
20


`items` is a list with 20 items.

In [37]:
print type(search_json['items'][0])
print type(search_json['items'][19])

<type 'dict'>
<type 'dict'>


Each of the 20 items in the list is a dictionary. 

In [38]:
first_item = search_json['items'][0]

print first_item.keys()

[u'sequence', u'county', u'edition', u'frequency', u'id', u'section_label', u'city', u'date', u'title', u'end_year', u'note', u'state', u'subject', u'type', u'place_of_publication', u'start_year', u'edition_label', u'publisher', u'language', u'alt_title', u'lccn', u'country', u'ocr_eng', u'batch', u'title_normal', u'url', u'place', u'page']


While a standard CSV file has a header row that describes the contents of each column, a JSON file has keys identifying the values found in each case. Importantly, these keys need not be the same for each item. Additionally, values don't have to be numbers of strings, but could be lists or dictionaries. For example, this JSON could have included a `newspaper` key that was a dictionary with all the metadata about the newspaper the article and issue was published, an `article` key that include the article specific information as another dictionary, and a `text` key whose value was a string with the article text.

As before, we can examine the contents of a particular item, such as the publication's `title`.

In [39]:
print first_item['title']

Anti-slavery bugle. volume


The easiest way to view or analyze this data is to convert it to a dataset-like structure. While Python does not have a builting in dataframe type, the popular `pandas` library does. By convention, it is imported as `pd`.

In [40]:
import pandas as pd

# Make sure all columns are displayed
pd.set_option("display.max_columns",101)

pandas is prety smart about importing different JSON-type objects and converting them to dataframes with its `.DataFrame()` function.

In [41]:
df = pd.DataFrame(search_json['items'])

df.head(10)

,alt_title,batch,city,country,county,date,edition,edition_label,end_year,frequency,id,language,lccn,note,ocr_eng,page,place,place_of_publication,publisher,section_label,sequence,start_year,state,subject,title,title_normal,type,url
0,[],batch_ohi_ariel_ver02,"[New Lisbon, Salem]",Ohio,"[Columbiana, Columbiana]",18490316,None,,1861,Weekly,/lccn/sn83035487/1849-03-16/ed-1/seq-1/,[English],sn83035487,[Archived issues are available in digital form...,"LAVE\nam\nJlile\nVOL. 4. NO. 30.\nSALEM. OHIO,...",,"[Ohio--Columbiana--New Lisbon, Ohio--Columbian...","New-Lisbon, Ohio",Ohio American Antislavery Society,,1,1845,"[Ohio, Ohio]",[Antislavery movements--United States--Newspap...,Anti-slavery bugle. volume,anti-slavery bugle.,page,http://chroniclingamerica.loc.gov/lccn/sn83035...
1,[],batch_iune_golf_ver01,[Chicago],Illinois,[Cook County],19140516,None,NOON EDITION,1917,Daily (except Sunday and holidays),/lccn/sn83045487/1914-05-16/ed-1/seq-10/,[English],sn83045487,"[""An adless daily newspaper."", Archived issues...",r\nmmmmmmmmmmmmmmmmmmmmmmmm\n'SLAVERY RIFE IN ...,,[Illinois--Cook County--Chicago],"Chicago, Ill.",N.D. Cochran,,10,1911,[Illinois],"[Chicago (Ill.)--Newspapers., Illinois--Chicag...",The day book.,day book.,page,http://chroniclingamerica.loc.gov/lccn/sn83045...
2,[],batch_iune_india_ver01,[Chicago],Illinois,[Cook County],19161109,None,EXTRA,1917,Daily (except Sunday and holidays),/lccn/sn83045487/1916-11-09/ed-1/seq-26/,[English],sn83045487,"[""An adless daily newspaper."", Archived issues...",us remaining whites if we expect to\nstay on t...,,[Illinois--Cook County--Chicago],"Chicago, Ill.",N.D. Cochran,,26,1911,[Illinois],"[Chicago (Ill.)--Newspapers., Illinois--Chicag...",The day book.,day book.,page,http://chroniclingamerica.loc.gov/lccn/sn83045...
3,[],batch_iune_golf_ver01,[Chicago],Illinois,[Cook County],19150327,None,NOON EDITION,1917,Daily (except Sunday and holidays),/lccn/sn83045487/1915-03-27/ed-1/seq-24/,[English],sn83045487,"[""An adless daily newspaper."", Archived issues...",THOUSANDS OF VEILED WOMEN OF TURKISH\nHAREM ON...,,[Illinois--Cook County--Chicago],"Chicago, Ill.",N.D. Cochran,,24,1911,[Illinois],"[Chicago (Ill.)--Newspapers., Illinois--Chicag...",The day book.,day book.,page,http://chroniclingamerica.loc.gov/lccn/sn83045...
4,[],batch_iune_foxtrot_ver01,[Chicago],Illinois,[Cook County],19130815,None,,1917,Daily (except Sunday and holidays),/lccn/sn83045487/1913-08-15/ed-1/seq-5/,[English],sn83045487,"[""An adless daily newspaper."", Archived issues...",LOLA NORRiajQlVS SiENSAT-iPN AL t EVIDENCE IN ...,,[Illinois--Cook County--Chicago],"Chicago, Ill.",N.D. Cochran,,5,1911,[Illinois],"[Chicago (Ill.)--Newspapers., Illinois--Chicag...",The day book.,day book.,page,http://chroniclingamerica.loc.gov/lccn/sn83045...
5,[],batch_iune_foxtrot_ver01,[Chicago],Illinois,[Cook County],19130308,None,NOON EDITION,1917,Daily (except Sunday and holidays),/lccn/sn83045487/1913-03-08/ed-1/seq-6/,[English],sn83045487,"[""An adless daily newspaper."", Archived issues...",that every possible weakness in. a\ngirl as &e...,,[Illinois--Cook County--Chicago],"Chicago, Ill.",N.D. Cochran,,6,1911,[Illinois],"[Chicago (Ill.)--Newspapers., Illinois--Chicag...",The day book.,day book.,page,http://chroniclingamerica.loc.gov/lccn/sn83045...
6,[],batch_iune_foxtrot_ver01,[Chicago],Illinois,[Cook County],19130424,None,,1917,Daily (except Sunday and holidays),/lccn/sn83045487/1913-04-24/ed-1/seq-13/,[English],sn83045487,"[""An adless daily newspaper."", Archived issues...",mpICFED FOR WHITE -SLAVERY.\nTop Lola Norris-a...,,[Illinois--Cook County--Chicago],"Chicago, Ill.",N.D. Cochran,,13,1911,[Illinois],"[Chicago (Ill.)--Newspapers., Illinois--Chicag...",The day book.,day book.,page,http://chroniclingamerica.loc.gov/lccn/sn83045...
7,[],batch_dlc_elf_ver03,[Washington],District of Columbia,[None],18540511,None,,1860,Weekly,/lccn/sn84026752/1854-05-11/ed-1/seq-1/,[English],sn84026752,[Also issued on microfilm by University Microf

Note that I converted `search_json['items']` to  dataframe and not the entire JSON file. This is because I wanted each row to be an article. 

If this dataframe contained all the items that you were looking for, it would be easy to save this to a csv file for storage and later analysis.

In [42]:
df.to_csv('lynching_articles.csv')

This is only a small subset of the articles on lynching that are available, however. The API returns results in batches of 20 and this is only the first page of results. As is often the case, I'll need to make multiple calls to the API to retrieve all the data of interest. The easiest way to do that is to define a small function for getting the article information and put that in a loop. While it isn't a requirement that you create a function for making the API call, it will make your code easier to read and debug.


Looking at the API guidelines, there is an additional paramater `page` that tells the API which subset of results we want. This name varies by API but their is usually some mechanism for retrieiving results beyond the initial JSON.

Before creating the loop and making multiple calls to the API, I want to make sure that the API is working the way I think it is. 

In [43]:
base_url = 'http://chroniclingamerica.loc.gov/search/pages/results/'
parameters = '?andtext=slavery&format=json&page=3'
r = requests.get(base_url + parameters)
results =  r.json()

print results['startIndex']
print results['endIndex']

41
60


A call to random selected page 3 returns results 41 through 60, which is what I expected since each page has 20 items.

The parameters are getting pretty ugly, so fortunately `requests` accepts a dictionary where the keys are the parameter names as defined by the API and the values are the search paramaters you are looking for. So the same request can be rewritten:

In [44]:
base_url = 'http://chroniclingamerica.loc.gov/search/pages/results/'
parameters = {'andtext': 'lynching',
              'page' : 3,
              'format'  : 'json'}
r = requests.get(base_url, params=parameters)

results =  r.json()

print results['startIndex']
print results['endIndex']

41
60


This can be rewritten as function:

In [45]:
def get_articles():
    '''
    Make calls to the Chronicling America API.
    '''
    base_url = 'http://chroniclingamerica.loc.gov/search/pages/results/'
    parameters = {'andtext': 'lynching',
                  'page' : 3,
              'format'  : 'json'}
    r = requests.get(base_url, params=parameters)
    results =  r.json()
    return results

In [46]:
results = get_articles()

print results['startIndex']
print results['endIndex']

41
60


The advantage of writing a function, however, would be that you can pass along your own parameters, such as the search term and page number, which would make this much more useful. 

In [47]:
def get_articles(search_term, page_number):
    '''
    Make calls to the Chronicling America API.
    '''
    base_url = 'http://chroniclingamerica.loc.gov/search/pages/results/'
    parameters = {'andtext': search_term,
                  'page' : page_number,
              'format'  : 'json'}
    r = requests.get(base_url, params=parameters)
    results =  r.json()
    return results

In [48]:
results = get_articles('lynching', 3)

print results['startIndex']
print results['endIndex']

41
60


Now, the first 60 results could downloaded in a just a few lines:

In [49]:
for page_number in range(1,4): # range stops before it gets to the last number
    results = get_articles('lynching', page_number)
    print results['startIndex'], results['endIndex']
    

1 20
21 40
41 60


Everything appears to be working, but unfortunately I only have the last page of results still. Each call to the API was redefining `results` variable. In this case, I set up an empty dataframe to store the results and will append the items from each page of results.

In [50]:
df = pd.DataFrame()

for page_number in range(1,4):
    results = get_articles('lynching', page_number)
    new_df = pd.DataFrame(results['items'])
    df = df.append(new_df , ignore_index=True)
    
print len(df)
df.head(5)

60


,alt_title,batch,city,country,county,date,edition,edition_label,end_year,frequency,id,language,lccn,note,ocr_eng,page,place,place_of_publication,publisher,section_label,sequence,start_year,state,subject,title,title_normal,type,url
0,[],batch_mimtptc_jackson_ver01,[Dearborn],Michigan,[Wayne],19211022,None,,1927,Weekly,/lccn/2013218776/1921-10-22/ed-1/seq-1/,[English],2013218776,"[""The Ford international weekly"" appears with ...","""Mis-Picturing Us Abroad"" Introduces the Serie...",,[Michigan--Wayne--Dearborn],"Dearborn, Mich.",Suburban Pub. Co.,,1,1901,[Michigan],"[Dearborn (Mich.)--Newspapers., Michigan--Dear...",Dearborn independent.,dearborn independent.,page,http://chroniclingamerica.loc.gov/lccn/2013218...
1,[],batch_iune_hotel_ver01,[Chicago],Illinois,[Cook County],19150818,None,LAST EDITION,1917,Daily (except Sunday and holidays),/lccn/sn83045487/1915-08-18/ed-1/seq-4/,[English],sn83045487,"[""An adless daily newspaper."", Archived issues...",25 patriots who took into their own\nhands a l...,,[Illinois--Cook County--Chicago],"Chicago, Ill.",N.D. Cochran,,4,1911,[Illinois],"[Chicago (Ill.)--Newspapers., Illinois--Chicag...",The day book.,day book.,page,http://chroniclingamerica.loc.gov/lccn/sn83045...
2,"[Star, Sunday star]",batch_dlc_dalek_ver01,[Washington],District of Columbia,[None],19221123,None,,1972,Daily,/lccn/sn83045462/1922-11-23/ed-1/seq-34/,[English],sn83045462,"[""From April 25 through May 24, 1861 one sheet...",T\nTU^\nnit;\n- V E\nxl\nII\nb<\nin rour\n3436...,34,[District of Columbia--Washington],"Washington, D.C.",W.D. Wallach & Hope,,34,1854,[District of Columbia],"[Washington (D.C.)--fast--(OCoLC)fst01204505, ...",Evening star.,evening star.,page,http://chroniclingamerica.loc.gov/lccn/sn83045...
3,[],batch_wa_elm_ver01,[Seattle],Washington,[King],19180608,None,,1921,Weekly,/lccn/sn87093353/1918-06-08/ed-1/seq-4/,[English],sn87093353,"[""A publication of general information, but in...",DAMNABLE WHITE WHELPS\nSome of the white subsc...,,[Washington--King--Seattle],"Seattle, Wash.",H.R. Cayton,,4,1916,[Washington],[African Americans--Washington (State)--Seattl...,Cayton's weekly.,cayton's weekly.,page,http://chroniclingamerica.loc.gov/lccn/sn87093...
4,[],batch_iune_delta_ver01,[Chicago],Illinois,[Cook County],19130507,None,,1917,Daily (except Sunday and holidays),/lccn/sn83045487/1913-05-07/ed-1/seq-12/,[English],sn83045487,"[""An adless daily newspaper."", Archived issues...",TEN COMPANIES OF MILITIA PROTECT ..THE\nALLEGE...,,[Illinois--Cook County--Chicago],"Chicago, Ill.",N.D. Cochran,,12,1911,[Illinois],"[Chicago (Ill.)--Newspapers., Illinois--Chicag...",The day book.,day book.,page,http://chroniclingamerica.loc.gov/lccn/sn83045...


For a large download, you would still want to tweak this a bit by pausing between each API call and making it robust to internet or API errors, but this is a solid framework for collecting data from an API.

### A second API

While the Chronicling America API allows annonymous usage, most APIs require you to register in advance. This usually involves going to their website, signing up for the service, and then going through a second signup for developers.  
When you sign up  to use an API, you usually agree to only use the API to facilitate other people using the service (e.g. customer's finding their way to your store) and that you won't store the data. API providers usually enforce this through rate limiting, meaning you can only access the service so many times per minute or per day. For example, you can only search status updates 180 times every 15 minutes according to [Twitter guidelines](https://dev.twitter.com/docs/rate-limiting/1.1/limits). [Yelp](http://www.yelp.com/developers/documentation/faq) limits you to 10,000 calls per day. If you go over your limit, you won't be able to access the service for a bit. You will also get in trouble if you redistribute the data, so don't plan on doing that. 

Two of the major reasons that web services require API authentication is so that they know who you are and so they can make sure that you don't go over their rate limits. Since you shouldn't be giving your password to random people on the internet, API authentication works a little bit differently. Like many other places, in order to use the Yelp API you have to sign up as [developer](http://www.yelp.com/developers). After telling them a little bit about what you plan to do--feel free to be honest; they aren't going to deny you access if you put "research on food cultures" as the purpose--you will get a Consumer Key, Consumer Secret, Token, and Token Secret. Copy and paste them somewhere special. 

Using the Yelp API goes something like this. First, you tell Yelp who you are and what you want. Assuming you are authorized to have this information, they respond with a URL where you can retrieve the data. The coding for this in practice is a little bit complicated, so there are often single use tools for accessing APIs, like [Tweepy](http://tweepy.github.io) for Twitter. 

Yelp uses the OAuth protocol for authentication. There are several python libraries for handling this, but you will likely need to install one (via `conda` or `pip`) yourself first.

In [51]:
import oauth2


There's no module to install for the Yelp API, but Yelp does provide some [sample Python code](https://github.com/Yelp/yelp-api/tree/master/v2/python). I've slightly modified the code below to show a sample search for restaurants near Chapel Hill, NC, sorted by distance. You can find more options in the search [documentation](http://www.yelp.com/developers/documentation/v2/search_api). The API's search options include things like location and type of business, and allows you to sort either by distance or popularity.



In [52]:
consumer_key    = 'qDBPo9c_szHVrZwxzo-zDw'
consumer_secret = '4we8Jz9rq5J3j15Z5yCUqmgDJjM'
token           = 'jeRrhRey_k-emvC_VFLGrlVHrkR4P3UF'
token_secret    = 'n-7xHNCxxedmAMYZPQtnh1hd7lI'

consumer = oauth2.Consumer(consumer_key, consumer_secret)

category_filter = 'restaurants'
location = 'Chapel Hill, NC'
options =  'category_filter=%s&location=%s&sort=1' % (category_filter, location)
url = 'http://api.yelp.com/v2/search?' + options

oauth_request = oauth2.Request('GET', url, {})
oauth_request.update({'oauth_nonce'      : oauth2.generate_nonce(),
                      'oauth_timestamp'  : oauth2.generate_timestamp(),
                      'oauth_token'       : token,
                      'oauth_consumer_key': consumer_key})

token = oauth2.Token(token, token_secret)
oauth_request.sign_request(oauth2.SignatureMethod_HMAC_SHA1(), consumer, token)
signed_url = oauth_request.to_url()

print signed_url

http://api.yelp.com/v2/search?sort=1&oauth_body_hash=2jmj7l5rSw0yVb%2FvlWAYkK%2FYBwk%3D&oauth_nonce=35980538&oauth_timestamp=1475698040&oauth_consumer_key=qDBPo9c_szHVrZwxzo-zDw&oauth_signature_method=HMAC-SHA1&category_filter=restaurants&oauth_token=jeRrhRey_k-emvC_VFLGrlVHrkR4P3UF&location=Chapel+Hill%2C+NC&oauth_signature=FVpomqjb96J3rG2Eb5bLmzaCSrs%3D


The URL returned expires after a couple of seconds, so don't expect for the above link to work. The results are provided in the JSON file format, so I'm going to use the already imported `requests` module to download them.

In [53]:
resp = requests.get(url=signed_url)
chapel_hill_restaurants = resp.json()

print chapel_hill_restaurants.keys()

[u'region', u'total', u'businesses']


As with the Chronacling America API, the top level of the JSON contains some metadata about the search with all the specific items in one field. In this case, `businesses`.

In [54]:
chapel_hill_restaurants['businesses'][1]

{u'categories': [[u'Burgers', u'burgers'], [u'Hot Dogs', u'hotdog']],
 u'display_phone': u'+1-919-240-4746',
 u'id': u'buns-chapel-hill',
 u'image_url': u'https://s3-media1.fl.yelpcdn.com/bphoto/5VCBOVZXf0y5TmIVTTuD1w/ms.jpg',
 u'is_claimed': True,
 u'is_closed': False,
 u'location': {u'address': [u'107 N Columbia St'],
  u'city': u'Chapel Hill',
  u'coordinate': {u'latitude': 35.913498068127,
   u'longitude': -79.056107631617},
  u'country_code': u'US',
  u'display_address': [u'107 N Columbia St', u'Chapel Hill, NC 27514'],
  u'geo_accuracy': 9.5,
  u'postal_code': u'27514',
  u'state_code': u'NC'},
 u'mobile_url': u'https://m.yelp.com/biz/buns-chapel-hill?adjust_creative=qDBPo9c_szHVrZwxzo-zDw&utm_campaign=yelp_api&utm_medium=api_v2_search&utm_source=qDBPo9c_szHVrZwxzo-zDw',
 u'name': u'Buns',
 u'phone': u'9192404746',
 u'rating': 4.0,
 u'rating_img_url': u'https://s3-media4.fl.yelpcdn.com/assets/2/www/img/c2f3dd9799a5/ico/stars/v1/stars_4.png',
 u'rating_img_url_large': u'https://s3

Inspecting the returned results for one restaraunt, it is clear that Yelp is keeping a lot of the review data for themselves. They returned the overall restaurant `rating`, but they provide only a small bit of text (`snippet_text`) instead of the full reviews and ratings. 



In [55]:
print chapel_hill_restaurants['total']
print len(chapel_hill_restaurants['businesses'])

40
20


Additionally, they cap the total number of business the search will return at 40 and only provide 20 results for each API call.

Even with these restrictions, it still might be useful for social science research. As before, you would likely want to define a function in order to make repeated calls to the API. In this, the easier solution might be to create two functions. One that gets a single page and another which retrieves both pages for a single geographical area by calling the first function twice. While it would be possible to do this with zero or one new functions, creating two functions allows for better control over finding and debugging errors since you can test each function independently. Creating lots of small functions generally the code more readable, especially in case like this where you are looping over pages within restaurants within geographic areas. In general, I think the principle of a workflow consisting of small functions, as is commonly found in Python code, is something that social scientists should adopt even when they aren't writing Python.

In [64]:
def get_yelp_page(location, offset):
    '''
    Retrieve one page of results from the Yelp API
    Returns a JSON file
    '''
    # from https://github.com/Yelp/yelp-api/tree/master/v2/python
    consumer_key    = 'qDBPo9c_szHVrZwxzo-zDw'
    consumer_secret = '4we8Jz9rq5J3j15Z5yCUqmgDJjM'
    token           = 'jeRrhRey_k-emvC_VFLGrlVHrkR4P3UF'
    token_secret    = 'n-7xHNCxxedmAMYZPQtnh1hd7lI'
    
    consumer = oauth2.Consumer(consumer_key, consumer_secret)
    
    url = 'http://api.yelp.com/v2/search?category_filter=restaurants&location=%s&sort=1&offset=%s' % (location, offset)
    
    oauth_request = oauth2.Request('GET', url, {})
    oauth_request.update({'oauth_nonce': oauth2.generate_nonce(),
                          'oauth_timestamp': oauth2.generate_timestamp(),
                          'oauth_token': token,
                          'oauth_consumer_key': consumer_key})
    
    token = oauth2.Token(token, token_secret)
    
    oauth_request.sign_request(oauth2.SignatureMethod_HMAC_SHA1(), consumer, token)
    
    signed_url = oauth_request.to_url()
    resp = requests.get(url=signed_url)
    return resp.json()

def get_yelp_results(location):
    '''
    Retrive both pages of results from the Yelp API
    Returns a dataframe
    '''
    df = pd.DataFrame()
    for offset in [1,21]:
        results = get_yelp_page(location, offset)
        new_df = pd.DataFrame(results['businesses'])
        df = df.append(new_df , ignore_index=True)
    return df

In [65]:
ch_df = get_yelp_results('Chapel Hill, NC')

print len(ch_df)

39


In [71]:
ch_df.keys()

Index([u'categories', u'display_phone', u'id', u'image_url', u'is_claimed',
       u'is_closed', u'location', u'menu_date_updated', u'menu_provider',
       u'mobile_url', u'name', u'phone', u'rating', u'rating_img_url',
       u'rating_img_url_large', u'rating_img_url_small', u'review_count',
       u'snippet_image_url', u'snippet_text', u'url'],
      dtype='object')

In [74]:
ch_df[['name','categories','review_count','rating']].sort_values(by='rating', ascending=False)

,name,categories,review_count,rating
5,Khushi Salads & Wraps,"[[Salad, salad], [Indian, indpak], [Vegan, veg...",16,5.0
38,Tarantini,"[[Italian, italian]]",67,4.5
8,Imbibe,"[[American (New), newamerican], [Wine Bars, wi...",15,4.5
18,Al's Burger Shack,"[[Burgers, burgers]]",257,4.5
23,Merritt's Store & Grill,"[[Sandwiches, sandwiches], [Convenience Stores...",219,4.5
24,Sunrise Biscuit Kitchen,"[[Breakfast & Brunch, breakfast_brunch]]",445,4.5
12,Mediterranean Deli,"[[Greek, greek], [Mediterranean, mediterranean...",549,4.5
32,Great Harvest Bread Co,"[[Sandwiches, sandwiches], [Bakeries, bakeries]]",30,4.5
31,Elements,"[[Asian Fusion, asianfusion], [American (New),...",97,4.5
33,Olio & Aceto Cafe,"[[Cafes, cafes]]",30,4.5


The function expects that the first thing you input will be a location. Taking advantage of both `oath2`'s ability to clean up the text so that it is functional when put in a URL (e.g., escape spaces) and Yelp's savvy ability to parse locations, the value for location can be fairly wide (e.g., "Chapel Hill" or "90210"). You can also add a category of business to search for from the [list](http://www.yelp.com/developers/documentation/category_list) of acceptable values. If you don't provide a value, `category_filter = 'restaurants'` provides a default value of 'restaurants'. This function returns the JSON formatted results. Note that this doesn't have any mechanism for handling errors, which will need to happen elsewhere.

In [58]:
chapel_hill_restaurants = get_yelp_businesses('Chapel Hill, NC', 21)
for business in chapel_hill_restaurants['businesses']:
    print '%s - %s (%s)' % (business['rating'], business['name'], business['review_count'])

NameError: name 'get_yelp_businesses' is not defined

In [ ]:
beverly_hills_restaurants = get_yelp_businesses('90210')
for business in beverly_hills_restaurants['businesses']:
    print '%s - %s (%s)' % (business['rating'], business['name'], business['review_count'])

In [59]:
pwd

u'/Users/nealcaren/Documents/Collecting-Text'